# Exercises
## Gradient Descent

In this exercise we will implement a routine to perform geometry optimizations using the gradient descent algorithm. This is the simplest optimization procedure and it requires knowledge only of the coordinates and gradient. 

Let's start by setting up a molecule, running an SCF calculation and setting up the gradient driver.

In [1]:
import veloxchem as vlx
import py3Dmol as p3d
from veloxchem.veloxchemlib import bohr_in_angstroms
import numpy as np
from matplotlib import pyplot as plt

In [2]:
basis_set_label = 'sto-3g'

ethene_xyz = """6
 ethene
 C          0.000000    -0.703984    0.000000
 C          0.000000     0.663984   0.000000
 H          0.919796    -1.223061   0.000000
 H         -0.919796    -1.223061   0.000000
 H          0.919796     1.223061   0.000000
 H         -0.919796     1.223061   0.000000
"""
molecule = vlx.Molecule.from_xyz_string(ethene_xyz)
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

In [3]:
view = p3d.view(linked=True, viewergrid=(1,1),width=500,height=300)
view.addModel(ethene_xyz, 'xyz', viewer=(0,0))
view.setStyle({'stick': {}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

```python
scf_settings = {'conv_thresh': 1.0e-10}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}

# SCF
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
```

                                                                                                                              
                                                Self Consistent Field Driver Setup                                            
                                               ====================================                                           
                                                                                                                              
                       Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                       Initial Guess Model             : Superposition of Atomic Densities                                    
                       Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                       Max. Number of Iterations       : 50                                                                   
                       Max. Number of Error Vectors    : 10                                                                   
                       Convergence Threshold           : 1.0e-10                                                              
                       ERI Screening Scheme            : Cauchy Schwarz + Density                                             
                       ERI Screening Mode              : Dynamic                                                              
                       ERI Screening Threshold         : 1.0e-12                                                              
                       Linear Dependence Threshold     : 1.0e-06                                                              
                                                                                                                              
    * Info * Nuclear repulsion energy: 33.1906218280 a.u.
                                                          ...
                                                          ...
                                                          ...

```python
# Set up a gradient driver
gradient_settings = {'numerical': 'no'} # yes or leave out

grad_driver = vlx.ScfGradientDriver(scf_drv=scfdrv)
grad_driver.update_settings(gradient_settings, method_settings)
grad_driver.compute(molecule, basis)
cart_grad_array = grad_driver.gradient
```

Now let's write a routine which runs one gradient descent iteration:

```python
def gradient_descent_iteration(coordinates, gradient, step):
        ...
    return new_coordinates
```

And the routine that runs the optimization:

```python
def gradient_descent(molecule, basis, scf_driver, gradient_driver,
                     step=0.1, threshold=1e-3, max_iter=10):

    # set ostream state to False, to avoid printout from every new scf calculation
    ostream_state = scf_driver.ostream.state
    scf_driver.ostream.state = False
    gradient_driver.ostream.state = False
    
    iteration = 0
    grad_norm = 100
    # atom labels (symbols)
    labels = molecule.get_labels()
    # initial atomc coordinates
    old_coords = molecule.get_coordinates()
    scf_driver.compute(molecule, basis, None)
    old_energy = scf_driver.get_scf_energy()
    grad_driver.compute(molecule, basis)
    old_gradient = gradient_driver.gradient
    
    print("Starting gradient descent:\n")
    print("Iteration     Energy (H)      Max. displacement (bohr)    Gradient norm (H/bohr)")

    energies = [old_energy]
    iterations = [0]

    while (grad_norm >= threshold) and (iteration <= max_iter):
        ...
        ...
        ...

        
    if iteration <= max_iter:
        print("\n   *** Gradient Descent converged in %d iteration(s). *** " % iteration)
        return new_mol, iterations, energies
    else:
        print("\n   !!! Gradient Descent did not converge  !!! ")
        return None, None, None

    # Set the ostream state to initial value
    scf_driver.ostream.state = ostream_state
    gradient_driver.ostream.state = ostream_state
```

```python
opt_mol, gd_iterations, gd_energies = gradient_descent(molecule, basis, scfdrv, grad_driver, threshold=5e-3, max_iter=100)
```


    Starting gradient descent:
    
    Iteration     Energy (H)      Max. displacement (bohr)    Gradient norm (H/bohr)
         0.      -77.0689063            0.0125735                0.1342139
         1.      -77.0705461            0.0105244                0.1100517
         2.      -77.0716458            0.0087580                0.0897577
        ...        ...                     ...                      ...


Let's look at the optimized geometry and compare it to the starting geometry:

```python
def get_xyz(molecule):
    natm = molecule.number_of_atoms()
    elements = molecule.get_labels()
    coords = molecule.get_coordinates() * bohr_in_angstroms()
    txt = "%d\n\n" % natm
    for i in range(natm):
        txt += elements[i] + " %15.7f %15.7f %15.7f\n" % (coords[i,0], coords[i,1], coords[i,2])
    return txt
```

In [4]:
opt_molecule_xyz = """6

C       0.0000000      -0.6632068       0.0000000
C       0.0000000       0.6431715       0.0000000
H       0.9220563      -1.2308716      -0.0000000
H      -0.9220563      -1.2308716      -0.0000000
H       0.9166605       1.2208893       0.0000000
H      -0.9166605       1.2208893       0.0000000"""

In [5]:
print("        Initial geometry:             Optimized geometry:")
view = p3d.view(linked=True, viewergrid=(1,2),width=500,height=300)
view.addModel(ethene_xyz, 'xyz', viewer=(0,0))
view.addModel(opt_molecule_xyz, 'xyz', viewer=(0,1))
view.setStyle({'stick': {}})
view.zoomTo()
view.show()

        Initial geometry:             Optimized geometry:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

How does the step size affect the convergence? Can you improve the convergence by using a step size which changes based on the gradient norm?

## Conjugate Gradient

The conjugate gradient algorithm improves upon the gradient descent by keeping track of two previous steps in the history of the optimization. Let's implement the conjugate gradient algorithm and see if and how it improves on the gradient descent algorithm.

```python
def conjugate_gradient_iteration(coordinates, h, step):
    ...
    return new_coordinates
```

```python
def conjugate_gradient(molecule, basis, scf_driver, gradient_driver,
                       step=0.1, threshold=1e-3, max_iter=10):
    # set ostream state to False, to avoid printout from every new scf calculation
    ostream_state = scf_driver.ostream.state
    scf_driver.ostream.state = False
    gradient_driver.ostream.state = False
    
    iteration = 0
    grad_norm = 100
    # atom labels (symbols)
    labels = molecule.get_labels()
    # initial atomc coordinates
    old_coords = molecule.get_coordinates()
    scf_driver.compute(molecule, basis, None)
    old_energy = scf_driver.get_scf_energy()
    grad_driver.compute(molecule, basis)
    old_gradient = gradient_driver.gradient
    old_h = old_gradient
    
    print("Starting gradient descent:\n")
    print("Iteration     Energy (H)      Max. displacement (bohr)    Gradient norm (H/bohr)      gamma")
    energies = [old_energy]
    iterations = [0]
    while (grad_norm >= threshold) and (iteration <= max_iter):
        
        ...
        ...
        ...
        
        gamma = grad_norm**2 / old_grad_norm**2
        
        print("   %3d.  %15.7f      %15.7f          %15.7f         %15.7f" % (iteration, energy, max_disp, grad_norm, gamma))

        h = gradient + gamma * old_h
        
        # save 
        old_energy = energy
        old_gradient = gradient
        old_coords = coords
        old_h = h
        iteration += 1
        iterations.append(iteration)
        
    if iteration <= max_iter:
        print("\n   *** Conjugate Gradient converged in %d iteration(s). *** " % iteration)
        return new_mol, iterations, energies
    else:
        print("\n   !!! Conjugate Gradient did not converge  !!! ")
        return None, None, None

    # restore ostream state to its original value
    scf_driver.ostream.state = ostream_state
    gradient_driver.ostream.state = ostream_state
```

```python
cg_opt_mol, cg_iterations, cg_energies = conjugate_gradient(molecule, basis, scfdrv, grad_driver, threshold=5e-3, max_iter=50)
```


    Starting gradient descent:
    
    Iteration     Energy (H)      Max. displacement (bohr)    Gradient norm (H/bohr)      gamma
         0.      -77.0689063            0.0125735                0.1342139               0.6819537
         1.      -77.0716492            0.0190990                0.0894813               0.4444977
         2.      -77.0730448            0.0157401                0.0519955               0.3376496
        ...         ...                   ...                      ...                     ...

### Comparison between Gradient Descent and Conjugate Gradient

```python
plt.figure(figsize=(7,4))

plt.plot(gd_iterations, gd_energies,'o-', label='Grad. Descent')
plt.plot(cg_iterations, cg_energies,'x-', label='Conj. Grad.')

plt.xlabel('Iteration')
plt.ylabel('Energy (H)')
plt.title("Gradient Descent vs. Conjugate Gradient")
plt.legend()
plt.tight_layout(); plt.show()
```

## Comparison to geomeTRIC

Run the same optimization using the veloxchem optimization driver which uses geomeTRIC to run the optimization steps. How do the gradient descent and conjugate gradient algorithms compare to geomeTRIC? Note that geomeTRIC uses the Newton-Raphons algorithm to take one optimization step. It offers different options for the type of coordinates used in the optimization: Cartesian, delocalized internal coordinates (DLC), as well as hybrid delocalized internal coordinates (HDLC). How does the choice of coordinates influence the optimization process?

```python
# Let's run the SCF and gradient again, to make sure we start from the same point:
scfdrv.ostream.state = False
grad_driver.ostream.state = False

scfdrv.compute(molecule, basis)
grad_driver.compute(molecule, basis)
```

```python
optimization_settings = {'coordsys' : 'cart'}
# 'tric': TRIC, default
# 'cart': Cartesian
# 'prim': Primitive Internal Coordinates
# 'dlc': Delocalized Internal Coordinates
# 'hdlc': Hybrid Delocalized Internal Coordinates
```

```python
opt_drv = vlx.OptimizationDriver(grad_driver)
opt_drv.update_settings(opt_dict=optimization_settings)
cart_opt_mol = opt_drv.compute(molecule, basis)
```


    geometric-optimize called with the following command line:
    /home/emi/.local/lib/python3.6/site-packages/ipykernel_launcher.py -f /home/emi/.local/share/jupyter/runtime/kernel-8ba05a83-4427-4dd6-988c-3936ee431e65.json
    
                                            [91m())))))))))))))))/[0m                     
                                        [91m())))))))))))))))))))))))),[0m                
                                    [91m*)))))))))))))))))))))))))))))))))[0m             
                            [94m#,[0m    [91m()))))))))/[0m                [91m.)))))))))),[0m          
                          [94m#%%%%,[0m  [91m())))))[0m                        [91m.))))))))*[0m        
                          [94m*%%%%%%,[0m  [91m))[0m              [93m..[0m              [91m,))))))).[0m      
                            [94m*%%%%%%,[0m         [93m***************/.[0m        [91m.)))))))[0m     
                    [94m#%%/[0m      [94m(%%%%%%,[0m    [93m/*********************.[0m       [91m)))))))[0m    
                  [94m.%%%%%%#[0m      [94m*%%%%%%,[0m  [93m*******/,[0m     [93m**********,[0m      [91m.))))))[0m   
                    [94m.%%%%%%/[0m      [94m*%%%%%%,[0m  [93m**[0m              [93m********[0m      [91m.))))))[0m  
              [94m##[0m      [94m.%%%%%%/[0m      [94m(%%%%%%,[0m                  [93m,******[0m      [91m/)))))[0m  
            [94m%%%%%%[0m      [94m.%%%%%%#[0m      [94m*%%%%%%,[0m    [92m,/////.[0m       [93m******[0m      [91m))))))[0m 
          [94m#%[0m      [94m%%[0m      [94m.%%%%%%/[0m      [94m*%%%%%%,[0m  [92m////////,[0m      [93m*****/[0m     [91m,)))))[0m 
        [94m#%%[0m  [94m%%%[0m  [94m%%%#[0m      [94m.%%%%%%/[0m      [94m(%%%%%%,[0m  [92m///////.[0m     [93m/*****[0m      [91m))))).[0m
      [94m#%%%%.[0m      [94m%%%%%#[0m      [94m/%%%%%%*[0m      [94m#%%%%%%[0m   [92m/////)[0m     [93m******[0m      [91m))))),[0m
        [94m#%%%%##%[0m  [94m%%%#[0m      [94m.%%%%%%/[0m      [94m(%%%%%%,[0m  [92m///////.[0m     [93m/*****[0m      [91m))))).[0m
          [94m##[0m     [94m%%%[0m      [94m.%%%%%%/[0m      [94m*%%%%%%,[0m  [92m////////.[0m      [93m*****/[0m     [91m,)))))[0m 
            [94m#%%%%#[0m      [94m/%%%%%%/[0m      [94m(%%%%%%[0m      [92m/)/)//[0m       [93m******[0m      [91m))))))[0m 
              [94m##[0m      [94m.%%%%%%/[0m      [94m(%%%%%%,[0m                  [93m*******[0m      [91m))))))[0m  
                    [94m.%%%%%%/[0m      [94m*%%%%%%,[0m  [93m**.[0m             [93m/*******[0m      [91m.))))))[0m  
                  [94m*%%%%%%/[0m      [94m(%%%%%%[0m   [93m********/*..,*/*********[0m       [91m*))))))[0m   
                    [94m#%%/[0m      [94m(%%%%%%,[0m    [93m*********************/[0m        [91m)))))))[0m    
                            [94m*%%%%%%,[0m         [93m,**************/[0m         [91m,))))))/[0m     
                          [94m(%%%%%%[0m   [91m()[0m                              [91m))))))))[0m       
                          [94m#%%%%,[0m  [91m())))))[0m                        [91m,)))))))),[0m        
                            [94m#,[0m    [91m())))))))))[0m                [91m,)))))))))).[0m          
                                     [91m()))))))))))))))))))))))))))))))/[0m             
                                        [91m())))))))))))))))))))))))).[0m                
                                             [91m())))))))))))))),[0m                     
    
    -=# [1;94m geomeTRIC started. Version: 0.9.7.2+148.g31b929c.dirty [0m #=-
    Custom engine selected.
    18 Cartesian coordinates being used
    Internal coordinate system (atoms numbered from 1):
    ...
    
    

Let's compare the Gradient Descent, Conjugate Gradient, and the Newton-Raphson routine using delocalized internal coordinates:

```python
plt.figure(figsize=(7,4))

plt.plot(gd_iterations, gd_energies,'o-', label='Grad. Descent')
plt.plot(cg_iterations, cg_energies,'x-', label='Conj. Grad.')
plt.plot(hdlc_iterations, hdlc_energies,'*-', label='HDLC Newton-Raphson')

plt.xlabel('Iteration')
plt.ylabel('Energy (H)')
plt.title("Gradient Descent vs. Conjugate Gradient vs. Newton-Raphson")
plt.legend()
plt.tight_layout(); plt.show()
```

Now let's compare the Newton-Raphson routine using different types of coordinates.

```python
plt.figure(figsize=(7,4))

plt.plot(cart_iterations, cart_energies,'o-', label='Cartesian Coordinates')
plt.plot(dlc_iterations, dlc_energies,'x-', label='DLC Coordinates')
plt.plot(hdlc_iterations, hdlc_energies,'*-', label='HDLC Coordinates')

plt.xlabel('Iteration')
plt.ylabel('Energy (H)')
plt.ylim(-77.075, -77.065)
plt.title("Newton-Raphson, different coordinate systems")
plt.legend()
plt.tight_layout(); plt.show()
```